# Terrible messy notebook...
..loading the JSON files to a pandas dataframe and then comparing the summary stats of the columns that should align with Mike's CSV. Extra cleaning on my end resolved the vast majority of points of conflit. Exported for manual review. (Do the aggregated columns make sense to aggregate? Notes on the persistent conflicts.)

In [489]:
import pandas,os
SAVEDIR = "data/cancelled_contracts"

## Mike isn't manually catching these and I don't know why he doesn't seem to have to
NA_STRS = ["N/A","NULL"]

dfs=[]
             
for file in os.listdir(SAVEDIR):
    if file[-4:] == "json":
        reason = file[-6]
        ndf = pandas.read_json(SAVEDIR+"/"+file)
        ndf["filedate"] = file.split("contracts-")[-1].split("_")[0]
        ndf["filename"] = file
        ndf["reason_code"] = reason
        dfs.append(ndf)

df = pandas.concat(dfs)
df = df.replace(NA_STRS, None)
df.describe().transpose().head(50)

fdf = df[df["reason_code"]=="F"]
fdf["filedate"] = pandas.to_datetime(fdf["filedate"])
fdf["relevantContractDates__effectiveDate"] = fdf[["content__award__relevantContractDates__effectiveDate",
                                                   "content__IDV__relevantContractDates__effectiveDate",
                                                   "content__OtherTransactionAward__contractDetail__relevantContractDates__effectiveDate"]].fillna('').sum(axis=1)

rosief = fdf[fdf["filedate"]<="2025-03-16"]

/tmp/ipykernel_110857/1759101205.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdf["filedate"] = pandas.to_datetime(fdf["filedate"])
/tmp/ipykernel_110857/1759101205.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdf["relevantContractDates__effectiveDate"] = fdf[["content__award__relevantContractDates__effectiveDate",


In [490]:
mikef = pandas.read_csv("data/collected_F-3-19.csv")
mikef = mikef[mikef["filedate"]<="2025-03-16"]
mikef_cols = pandas.Series(mikef.columns)
mikef_cols_temp = mikef_cols[mikef_cols.str.contains("placeOfPerformance")]
mikef.groupby("contract_type")[mikef_cols_temp].nunique()#.describe().head().transpose().head(50)#["dollarValues__obligatedAmount"].sum()

/tmp/ipykernel_110857/1934416055.py:1: DtypeWarning: Columns (13,25,27,43,139,143) have mixed types. Specify dtype option on import or set low_memory=False.
  mikef = pandas.read_csv("data/collected_F-3-19.csv")


,placeOfPerformanceZIPCode,placeOfPerformanceZIPCode__county,placeOfPerformanceZIPCode__city,placeOfPerformanceCongressionalDistrict
contract_type,,,,
AWARD,2655,482,901,52
IDV,0,0,0,0
OTHERTRANSACTIONAWARD,1,1,1,1


In [491]:
## Cleaning for better alignment!

def summary_sum(s):
    try:
        pandas.to_numeric(s).sum()
    except:
        return 0

def agg(row,cols,method):
    row[cols].agg(method)

def count_nunique(cols,df):
    ls = []
    for col in cols:
        ls += df[col].to_list()
    return pandas.Series(ls).nunique()


## Maybe could have avoided all of this by making them all strings to start out with
def make_str_coerce_floats_to_int(messy_str,zfill=None):
    if pandas.isna(messy_str):
        return None
    else:
        temp = str(messy_str).split(".")[0]
        if zfill:
            return temp.zfill(zfill)
        return temp
    
rosief["content__award__awardID__awardContractID__agencyID"] = rosief["content__award__awardID__awardContractID__agencyID"].apply(make_str_coerce_floats_to_int)
mikef["awardContractID__agencyID"] = mikef["awardContractID__agencyID"].apply(make_str_coerce_floats_to_int)

rosief[['content__award__purchaserInformation__contractingOfficeAgencyID',
        'content__IDV__purchaserInformation__contractingOfficeAgencyID',
        'content__OtherTransactionAward__contractDetail__purchaserInformation__contractingOfficeAgencyID']] = rosief[['content__award__purchaserInformation__contractingOfficeAgencyID',
                                                                                                                      'content__IDV__purchaserInformation__contractingOfficeAgencyID',
                                                                                                                      'content__OtherTransactionAward__contractDetail__purchaserInformation__contractingOfficeAgencyID']].map(make_str_coerce_floats_to_int)
mikef["contractingOfficeAgencyID"] = mikef["contractingOfficeAgencyID"].apply(make_str_coerce_floats_to_int)

rosief[['content__award__purchaserInformation__contractingOfficeAgencyID__departmentID',
        'content__IDV__purchaserInformation__contractingOfficeAgencyID__departmentID',
        'content__OtherTransactionAward__contractDetail__purchaserInformation__contractingOfficeAgencyID__departmentID']] = rosief[['content__award__purchaserInformation__contractingOfficeAgencyID__departmentID',
                                                                                                                                    'content__IDV__purchaserInformation__contractingOfficeAgencyID__departmentID',
                                                                                                                                    'content__OtherTransactionAward__contractDetail__purchaserInformation__contractingOfficeAgencyID__departmentID']].map(make_str_coerce_floats_to_int)
mikef['contractingOfficeAgencyID__departmentID'] = mikef['contractingOfficeAgencyID__departmentID'].apply(make_str_coerce_floats_to_int)

rosief["content__award__placeOfPerformance__placeOfPerformanceZIPCode"] = rosief["content__award__placeOfPerformance__placeOfPerformanceZIPCode"].apply(lambda x: make_str_coerce_floats_to_int(x,zfill=9))
rosief["content__OtherTransactionAward__contractDetail__placeOfPerformance__placeOfPerformanceZIPCode"] = rosief["content__OtherTransactionAward__contractDetail__placeOfPerformance__placeOfPerformanceZIPCode"].apply(lambda x: make_str_coerce_floats_to_int(x,zfill=9))
mikef["placeOfPerformanceZIPCode"] = mikef["placeOfPerformanceZIPCode"].apply(lambda x: make_str_coerce_floats_to_int(x,zfill=9))

rosief[['content__award__vendor__vendorSiteDetails__vendorLocation__ZIPCode',
         'content__IDV__vendor__vendorSiteDetails__vendorLocation__ZIPCode',
         'content__OtherTransactionAward__contractDetail__vendor__vendorSiteDetails__vendorLocation__ZIPCode']] = rosief[['content__award__vendor__vendorSiteDetails__vendorLocation__ZIPCode',
                                                                                                                             'content__IDV__vendor__vendorSiteDetails__vendorLocation__ZIPCode',
                                                                                                                             'content__OtherTransactionAward__contractDetail__vendor__vendorSiteDetails__vendorLocation__ZIPCode']].map(make_str_coerce_floats_to_int,zfill=9)

mikef["vendorLocation__ZIPCode"] = mikef["vendorLocation__ZIPCode"].apply(lambda x: make_str_coerce_floats_to_int(x,zfill=9))

rosief[['content__award__vendor__vendorSiteDetails__vendorLocation__phoneNo',
         'content__IDV__vendor__vendorSiteDetails__vendorLocation__phoneNo',
         'content__OtherTransactionAward__contractDetail__vendor__vendorSiteDetails__vendorLocation__phoneNo']] = rosief[['content__award__vendor__vendorSiteDetails__vendorLocation__phoneNo',
                                                                                                                         'content__IDV__vendor__vendorSiteDetails__vendorLocation__phoneNo',
                                                                                                                         'content__OtherTransactionAward__contractDetail__vendor__vendorSiteDetails__vendorLocation__phoneNo']].map(make_str_coerce_floats_to_int)
mikef["vendorLocation__phoneNo"] = mikef["vendorLocation__phoneNo"].apply(make_str_coerce_floats_to_int)


In [492]:
def list_vals(coln,df2):
    df2_cols = pandas.Series(df2.columns)
    relevant = df2_cols[df2_cols.str.endswith(coln)]
    rel_ls = relevant.to_list()
    return rel_ls
    
mikef_col_df = pandas.DataFrame(mikef_cols)
mikef_col_df.columns = ["column_name"]
mikef_col_df["aggregated_cols"] = mikef_col_df["column_name"].apply(lambda x: list_vals(x,rosief))
mikef_col_df["count_agg_cols"] = mikef_col_df["aggregated_cols"].apply(len)
mikef_col_df[mikef_col_df["count_agg_cols"]==4]["aggregated_cols"].loc[8]

mikef_col_df["mike_count"] = mikef_col_df["column_name"].apply(lambda x: mikef[x].count())
mikef_col_df["mike_sum"] = mikef_col_df["column_name"].apply(lambda x: summary_sum(x))
mikef_col_df["mike_nunique"] = mikef_col_df["column_name"].apply(lambda x: mikef[x].nunique())

mikef_col_df["rosie_count"] = mikef_col_df.apply(lambda x: rosief[x["aggregated_cols"]].count().sum(),axis=1)
mikef_col_df["rosie_sum"] = mikef_col_df.apply(lambda x: rosief[x["aggregated_cols"]].apply(summary_sum).sum(),axis=1)
mikef_col_df["rosie_nunique"] = mikef_col_df["aggregated_cols"].apply(lambda x: count_nunique(x,rosief))

mikef_col_df["counts_match"] = mikef_col_df["mike_count"] == mikef_col_df["rosie_count"]
mikef_col_df["sums_match"] = mikef_col_df["mike_sum"] == mikef_col_df["rosie_sum"]
mikef_col_df["nunique_match"] = mikef_col_df["mike_nunique"] == mikef_col_df["rosie_nunique"]

mikef_col_df.to_csv("data/processed/column_review.csv")

In [494]:
rosief["content__award__awardID__awardContractID__agencyID"] = rosief["content__award__awardID__awardContractID__agencyID"].apply(make_str_coerce_floats_to_int)
mikef["awardContractID__agencyID"] = mikef["awardContractID__agencyID"].apply(make_str_coerce_floats_to_int)

mikef_col_df[~mikef_col_df["counts_match"]]#["aggregated_cols"].iloc[-1]

,column_name,aggregated_cols,count_agg_cols,mike_count,mike_sum,mike_nunique,rosie_count,rosie_sum,rosie_nunique,counts_match,sums_match,nunique_match
2,reasoncode,[],0,20626,0,1,0,0.0,0,False,True,False
8,relevantContractDates__effectiveDate,[content__award__relevantContractDates__effect...,4,20626,0,2078,41252,0.0,2078,False,True,True
47,content__award__contractData__solicitationID,[content__award__contractData__solicitationID],1,0,0,0,3257,0.0,1979,False,True,False


# The rest of this is junk/scratch notes!

In [390]:
rosie_zips = pandas.concat([rosief["content__award__placeOfPerformance__placeOfPerformanceZIPCode"],rosief["content__OtherTransactionAward__contractDetail__placeOfPerformance__placeOfPerformanceZIPCode"]])
rosie_zips.value_counts()

temp = pandas.concat([mikef["placeOfPerformanceZIPCode"].value_counts(),rosie_zips.value_counts()], axis=1)
temp.columns = ["m","r"]
temp[temp["m"]!=temp["r"]]

,m,r


In [391]:

#df['combined'] = df['col1'].fillna(df['col2'])

rosief["combine_zip"] = rosief['content__award__placeOfPerformance__placeOfPerformanceZIPCode'].fillna(rosief['content__OtherTransactionAward__contractDetail__placeOfPerformance__placeOfPerformanceZIPCode'])
rosief["temp_zip_col"] = rosief["combine_zip"].apply(lambda x: str(x).split(".")[0]).str.zfill(9)
mikef["temp_zip_col"] = mikef["placeOfPerformanceZIPCode"].apply(lambda x: str(x).split(".")[0]).str.zfill(9)

zip_temp = pandas.concat([rosief["temp_zip_col"].value_counts(),mikef["temp_zip_col"].value_counts()],axis=1)
zip_temp.columns = ["rosie_count","mike_count"]
zip_temp[zip_temp["rosie_count"]!=zip_temp["mike_count"]]

,rosie_count,mike_count
temp_zip_col,,


In [479]:
mikef_col_df[~mikef_col_df["nunique_match"]]["aggregated_cols"].iloc[-1]

['content__award__vendor__vendorSiteDetails__vendorLocation__phoneNo',
 'content__IDV__vendor__vendorSiteDetails__vendorLocation__phoneNo',
 'content__OtherTransactionAward__contractDetail__vendor__vendorSiteDetails__vendorLocation__phoneNo']

In [440]:
rosief["temp_contract_off_id"] = rosief[['content__award__purchaserInformation__contractingOfficeAgencyID',
                                      'content__IDV__purchaserInformation__contractingOfficeAgencyID',
                                      'content__OtherTransactionAward__contractDetail__purchaserInformation__contractingOfficeAgencyID']].fillna("").astype(str).sum(axis=1)

rosief["temp_contract_off_id"] = rosief["temp_contract_off_id"].apply(make_str_coerce_floats_to_int)
rosief.groupby("contract_type")["temp_contract_off_id"].nunique()

contract_type
AWARD                    122
IDV                       45
OTHERTRANSACTIONAWARD      1
Name: temp_contract_off_id, dtype: int64

In [459]:
rosief["temp_contract_off_id"] = rosief["temp_contract_off_id"].apply(make_str_coerce_floats_to_int)
mikef.groupby("contract_type")["contractingOfficeAgencyID"].nunique()

contract_type
AWARD                    122
IDV                       45
OTHERTRANSACTIONAWARD      1
Name: contractingOfficeAgencyID, dtype: int64

In [458]:
mikef[mikef["contractingOfficeAgencyID"].isin(rosief["temp_contract_off_id"])==False]["contractingOfficeAgencyID"].value_counts()

contractingOfficeAgencyID
4732    4096
Name: count, dtype: int64

In [449]:
mikef[mikef["contractingOfficeAgencyID"]=="4732"].head().transpose().head(50)[0].title

'DELIVERY ORDER 47QSSC25F3UL7 (P00001) awarded to JLWS ENTERPRISES INCORPORATED, was modified for the amount of -$804.05'

In [456]:
TITLE = 'DELIVERY ORDER 47QSSC25F3UL7 (P00001) awarded to JLWS ENTERPRISES INCORPORATED, was modified for the amount of -$804.05'

rosief[(rosief["filename"]=="contracts-2025-01-20_F.json")&(rosief["title"]==TITLE)]["temp_contract_off_id"]

0    4732
Name: temp_contract_off_id, dtype: object

In [457]:
4732 in rosief["temp_contract_off_id"]

True

In [432]:
rosief["temp_contract_off_id"].value_counts()

temp_contract_off_id
4732    13505
97AS     2239
7523      540
3600      447
7200      249
        ...  
6947        1
1145        1
4730        1
4704        1
1549        1
Name: count, Length: 126, dtype: int64

In [244]:
rosief.groupby("contract_type")[['content__award__vendor__vendorSiteDetails__vendorLocation__state',
 'content__IDV__vendor__vendorSiteDetails__vendorLocation__state',
 'content__OtherTransactionAward__contractDetail__vendor__vendorSiteDetails__vendorLocation__state']].count()

rosief["temp_vendor_state"] = rosief[['content__award__vendor__vendorSiteDetails__vendorLocation__state',
 'content__IDV__vendor__vendorSiteDetails__vendorLocation__state',
 'content__OtherTransactionAward__contractDetail__vendor__vendorSiteDetails__vendorLocation__state']].fillna('').sum(axis=1)

rosief["temp_vendor_state"].value_counts()

state_temp = pandas.concat([rosief["temp_vendor_state"].value_counts(),mikef["vendorLocation__state"].value_counts()],axis=1)
state_temp.columns = ["rosie_count","mike_count"]
state_temp[state_temp["rosie_count"]!=state_temp["mike_count"]]

,rosie_count,mike_count
,131,NaN


In [254]:
mikef["content__award__contractData__solicitationID"].unique()

array(['nan'], dtype=object)

In [255]:
rosief["content__award__contractData__solicitationID"].unique()

array(['nan', '47QSMD20R0001', '47QSSC25Q1705', ..., 'SPRDL117Q0310',
       'W56ZTN19R0005', 'W91ZLK24R0018'], shape=(1981,), dtype=object)